# Forecasting

In [8]:
import pandas as pd
import numpy as np

In [9]:
# import the demand data from table 1 
# Load Table 1: Sales data (for calculating min_demand)
table1 = pd.read_csv('data/table1.csv')

# Optimizaiton

In [10]:
from docplex.mp.model import Model

In [11]:
# ------------------------------------------------------------------
# 1. DATA LOADING (from CSV files in data folder)
# ------------------------------------------------------------------

# Load Table 1: Sales data (for calculating min_demand)
table1 = pd.read_csv('data/table1.csv')

# Load Table 4: Prices, Profits, and Production Time
table4 = pd.read_csv('data/table4.csv')

# Load Table 5: Ingredients for each bread type
table5 = pd.read_csv('data/table5.csv')

# ------------------------------------------------------------------
# 2. DATA PROCESSING
# ------------------------------------------------------------------

# Extract product names from Table 4
products = table4['Product'].tolist()

# Goal-1 coefficients (unit profit, IDR) from Table 4
profit = table4['Profit (IDR)'].tolist()

# Goal-2 coefficients (production time, minutes) from Table 4
time_use = table4['Time Production (Minutes)'].tolist()

# Goal-3 ingredient matrix from Table 5
# Extract ingredient requirements for each product (columns 2-8)
ingredient_cols = [col for col in table5.columns if '(g)' in col and col != 'Stock (g)']
ingmt = []
for idx, row in table5.iterrows():
    ingredient_row = []
    for col in ingredient_cols:
        ingredient_row.append(row[col])
    ingmt.append(ingredient_row)

# Stocks (grams) for each ingredient from Table 5
stock = table5['Stock (g)'].tolist()

# Minimum-demand floor for each product
# (These values come from triple exponential smoothing - hardcoded for now)
min_demand = [829, 331, 549, 200, 437, 6789, 4627]

print("Data loaded successfully!")
print(f"Products: {len(products)}")
print(f"Ingredients: {len(stock)}")
print(f"Profit coefficients: {profit}")
print(f"Time coefficients: {time_use}")
print(f"Min demand: {min_demand}")

# ------------------------------------------------------------------
# 2. MODEL SETUP
# ------------------------------------------------------------------
mdl = Model('Rotte_Preemptive_GP')

# Decision vars: production qty for each of 7 breads
x = mdl.continuous_var_dict(range(7), lb=0, name='x')

# Deviation vars d⁺/d⁻ for 19 goals
d_plus  = mdl.continuous_var_dict(range(19), lb=0, name='d_plus')
d_minus = mdl.continuous_var_dict(range(19), lb=0, name='d_minus')

# Goal-1: profit balance  (d1+ / d1−)
mdl.add_constraint(
    mdl.sum(profit[i] * x[i] for i in range(7))
    - d_plus[0] + d_minus[0]
    == 32_000_000,
    'profit_goal'
)

# Goal-2: time balance (d2+ / d2−)
mdl.add_constraint(
    mdl.sum(time_use[i] * x[i] for i in range(7))
    - d_plus[1] + d_minus[1]
    == 24_000,
    'time_goal'
)

# Goal-3: ingredient balances (17 rows)
for k in range(17):
    mdl.add_constraint(
        mdl.sum(ingmt[k][i] * x[i] for i in range(7))
        - d_plus[2+k] + d_minus[2+k]
        == stock[k],
        f'ingred_{k+1}'
    )

# Minimum production (forecast)
for i in range(7):
    mdl.add_constraint(x[i] >= min_demand[i], f'min_demand_{i+1}')

# ------------------------------------------------------------------
# 3. MULTI-OBJECTIVE (lexicographic)
# ------------------------------------------------------------------
# Objectives:
#   obj1 = d_minus[0]                     # unmet profit
#   obj2 = d_plus[1]                      # overtime
#   obj3 = sum(d_plus[2]..d_plus[18])     # ingredient over-use
obj1 = d_minus[0]
obj2 = d_plus[1]
obj3 = mdl.sum(d_plus[j] for j in range(2, 19))

# Priorities: larger number = higher importance
p1, p2, p3 = 3, 2, 1

mdl.set_multi_objective(
    sense="min",
    exprs       =[obj1,      obj2,       obj3],
    priorities  =[p1,        p2,         p3],
    weights     =[1.0,       1.0,        1.0],
    abstols     = None,
    reltols     = None,
    names       = None
)

# ------------------------------------------------------------------
# 4. SOLVE & REPORT
# ------------------------------------------------------------------
sol = mdl.solve(log_output=True)
if sol:
    print("\nLexicographic objective values:")
    print(f"  Phase-1 unmet-profit  = {obj1.solution_value:,.0f}")
    print(f"  Phase-2 overtime      = {obj2.solution_value:,.0f}")
    print(f"  Phase-3 over-use sum  = {obj3.solution_value:,.0f}\n")
    print("Optimal production quantities:")
    for i, name in enumerate(products):
        print(f"  {name:<24}: {x[i].solution_value:,.0f} pcs")
else:
    print("No feasible solution found.")

Data loaded successfully!
Products: 7
Ingredients: 17
Profit coefficients: [5967, 4789, 3205, 2969, 3260, 1155, 1336]
Time coefficients: [4.3, 2.4, 0.8, 0.8, 0.8, 0.9, 0.6]
Min demand: [829, 331, 549, 200, 437, 6789, 4627]
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1

Multi-objective solve log . . .

Index Priority Blend Iterations            Objective Time (sec.) DetTime (ticks)
    1        3     1          0             0.000000        0.01            0.04
    2        2     1          0             0.000000        0.01            0.05
    3        1     1          8           349.524800        0.01            0.07

Lexicographic objective values:
  Phase-1 unmet-profit  = 0
  Phase-2 overtime      = 0
  Phase-3 over-use sum  = 350

Optimal production quantities:
  Plain Bread             : 829 pcs
  Coconut Pillow Bread    : 331 pcs
  Sausage Bread           : 667 pcs
  Cheese Floss Bread      : 333 pcs
  Mexicana Banana B